# Explore AIMed Dataset


#### Download AIMed dataset from ftp://ftp.cs.utexas.edu/pub/mooney/bio-data/interactions.tar.gz"



In [1]:
data_dir = "/users/aeg/Downloads/interactions 3"

In [2]:
import sys
sys.path.append('./source')

#### Covert to pandas

In [3]:
from datatransformer.AimedToDataFrame import AimedToDataFrame
df = AimedToDataFrame().load_dir(data_dir)


AssertionError: interactions 3 must be a directory

In [ ]:
df.head(n=10)

In [ ]:
df.shape

In [ ]:
unique_passages = df["passage"].unique()
unique_sentence_count=[ len(r.split("\n")) for r in unique_passages]
print("Unique setences :  " , sum(unique_sentence_count))

### Explore dataset

In [ ]:
import matplotlib.pyplot as plt

df["isValid"].value_counts().plot.pie(autopct='%.2f',figsize=(4, 4))
plt.show()

In [ ]:
df["isValid"].value_counts()

## Split into train/test/validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

unique_docids = df.docid.unique()
stratified =  [ df.query("docid == '{}'".format(p))['isValid'].iloc[0] for p in unique_docids]
traindocid, valdocid = train_test_split(unique_docids,  test_size=.1,
                                                        random_state=777, stratify=stratified)

In [ ]:

stratified =  [df.query("docid == '{}'".format(p))['isValid'].iloc[0] for p in traindocid]
traindocid, testdocid = train_test_split(traindocid,  test_size=.2,
                                                        random_state=777, stratify=stratified)

In [ ]:
train = df[df['docid'].isin(traindocid)]
test = df[df['docid'].isin(testdocid)]
val = df[df['docid'].isin(valdocid)]




In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,3, figsize=(15,20))

ax[0].set_title('Train class distribution')
train.isValid.value_counts().plot.pie(autopct='%.2f',  ax=ax[0])

ax[1].set_title('Validation class distribution')
val.isValid.value_counts().plot.pie(autopct='%.2f',  ax=ax[1])


ax[2].set_title('Test class distribution')
test.isValid.value_counts().plot.pie(autopct='%.2f',  ax=ax[2])


plt.show()

In [ ]:
train.to_json("AIMedtrain.json")
test.to_json("AIMedtest.json")
val.to_json("AIMedval.json")

In [ ]:
train.sample(n=20).to_json("Aimedsample.json")